### Генерация поэзии с помощью нейронных сетей: шаг 1
##### Автор: [Радослав Нейчев](https://www.linkedin.com/in/radoslav-neychev/), @neychev

Ваша основная задача: научиться генерироват стихи с помощью простой рекуррентной нейронной сети (Vanilla RNN). В качестве корпуса текстов для обучения будет выступать роман в стихах "Евгений Онегин" Александра Сергеевича Пушкина.

In [1]:
# do not change the code in the block below
# __________start of block__________
import string
import os
from random import sample

import numpy as np
import torch, torch.nn as nn
import torch.nn.functional as F

from IPython.display import clear_output

import matplotlib.pyplot as plt
# __________end of block__________

In [2]:
# do not change the code in the block below
# __________start of block__________
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('{} device is available'.format(device))
# __________end of block__________

cuda device is available


#### 1. Загрузка данных.

In [3]:
# do not change the code in the block below
# __________start of block__________
!wget https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/onegin.txt

with open('onegin.txt', 'r') as iofile:
    text = iofile.readlines()

text = "".join([x.replace('\t\t', '').lower() for x in text])
# __________end of block__________

--2024-11-04 19:48:38--  https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/onegin.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 262521 (256K) [text/plain]
Saving to: 'onegin.txt'

onegin.txt          100%[===================>] 256.37K  --.-KB/s    in 0.02s   

2024-11-04 19:48:39 (10.7 MB/s) - 'onegin.txt' saved [262521/262521]



#### 2. Построение словаря и предобработка текста
В данном задании требуется построить языковую модель на уровне символов. Приведем весь текст к нижнему регистру и построим словарь из всех символов в доступном корпусе текстов. Также добавим токен `<sos>`.

In [4]:
# do not change the code in the block below
# __________start of block__________
tokens = sorted(set(text.lower())) + ['<sos>']
num_tokens = len(tokens)

assert num_tokens == 84, "Check the tokenization process"

token_to_idx = {x: idx for idx, x in enumerate(tokens)}
idx_to_token = {idx: x for idx, x in enumerate(tokens)}

assert len(tokens) == len(token_to_idx), "Mapping should be unique"

print("Seems fine!")


text_encoded = [token_to_idx[x] for x in text]
# __________end of block__________

Seems fine!


__Ваша задача__: обучить классическую рекуррентную нейронную сеть (Vanilla RNN) предсказывать следующий символ на полученном корпусе текстов и сгенерировать последовательность длины 100 для фиксированной начальной фразы.

Вы можете воспользоваться кодом с занятие №6 или же обратиться к следующим ссылкам:
* Замечательная статья за авторством Andrej Karpathy об использовании RNN: [link](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
* Пример char-rnn от Andrej Karpathy: [github repo](https://github.com/karpathy/char-rnn)
* Замечательный пример генерации поэзии Шекспира: [github repo](https://github.com/spro/practical-pytorch/blob/master/char-rnn-generation/char-rnn-generation.ipynb)

Данное задание является достаточно творческим. Не страшно, если поначалу оно вызывает затруднения. Последняя ссылка в списке выше может быть особенно полезна в данном случае.

Далее для вашего удобства реализована функция, которая генерирует случайный батч размера `batch_size` из строк длиной `seq_length`. Вы можете использовать его при обучении модели.

In [5]:
# do not change the code in the block below
# __________start of block__________
batch_size = 256
seq_length = 100
start_column = np.zeros((batch_size, 1), dtype=int) + token_to_idx['<sos>']

def generate_chunk():
    global text_encoded, start_column, batch_size, seq_length

    start_index = np.random.randint(0, len(text_encoded) - batch_size*seq_length - 1)
    data = np.array(text_encoded[start_index:start_index + batch_size*seq_length]).reshape((batch_size, -1))
    yield np.hstack((start_column, data))
# __________end of block__________

Пример батча:

In [6]:
next(generate_chunk())

array([[83, 63, 45, ...,  1, 62, 64],
       [83, 56, 53, ..., 62, 58, 45],
       [83, 57,  5, ..., 60, 61, 59],
       ...,
       [83, 59, 68, ...,  0, 53, 52],
       [83, 59, 46, ..., 48, 59, 47],
       [83, 45, 76, ..., 49, 59, 54]])

Далее вам предстоит написать код для обучения модели и генерации текста.

In [7]:
class PoetryRecurrentUnit(nn.Module):
    def __init__(self, input_size, output_size, hidden_size=100, n_layers=4):
        super(PoetryRecurrentUnit, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, data, hidden):
        data = self.encoder(data.view(1, -1))
        data, hidden = self.gru(data, hidden)
        data = self.decoder(data).view(hidden.shape[1], -1)
        return data, hidden
    
    def predict(self, data, hidden):
        hidden_c, hidden_h = hidden
        data, hidden_c, hidden_h = data.to(device), hidden_c.to(device), hidden_h.to(device)
        output, hidden = self.forward(data, (hidden_c, hidden_h))
        return output, hidden
    
    def init_hidden(self, size):
        return torch.zeros(self.n_layers, size, self.hidden_size)

In [8]:
model = PoetryRecurrentUnit(num_tokens, num_tokens, n_layers=1)

model.to(device)

num_epochs = 2100
print_every = 100
plot_every = 10

optim = torch.optim.Adam(model.parameters(), lr=1e-2)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, factor=0.3, patience=10)
criterion = nn.CrossEntropyLoss()

In [9]:
def train_char_rnn(inps, targets):
    hidden = model.init_hidden(batch_size)
    hidden = hidden.to(device)
    model.zero_grad()
    loss = 0.
    
    for c in range(seq_length):
        inp, target = torch.tensor(inps[:, c]), torch.tensor(targets[:, c])
        inp, target = inp.to(device), target.to(device)
        output, hidden = model(inp, hidden)
        loss += criterion(output, target)
    loss.backward()
    optim.step()

    return loss.item() / seq_length

In [10]:
from tqdm.auto import tqdm as tqdma

In [11]:
def generate_sample(char_rnn, seed_phrase=None, max_length=200, temperature=1.0, device=device):
    '''
    The function generates text given a phrase of length at least SEQ_LENGTH.
    :param seed_phrase: prefix characters. The RNN is asked to continue the phrase
    :param max_length: maximum output length, including seed_phrase
    :param temperature: coefficient for sampling.  higher temperature produces more chaotic outputs,
                        smaller temperature converges to the single most likely output
    '''

    if seed_phrase is not None:
        x_sequence = [token_to_idx['<sos>']] + [token_to_idx[token] for token in seed_phrase]
    else:
        x_sequence = [token_to_idx['<sos>']]

    x_sequence = torch.tensor([x_sequence], dtype=torch.int64).to(device)

    #feed the seed phrase, if any

    hidden = model.init_hidden(1)
    hidden = hidden.to(device)
    
    for p in range(1, len(x_sequence[0]) - 1):
        _, hidden = model(x_sequence[0][p], hidden)
    
    inp = x_sequence[0][-1]
    
    for p in range(max_length - len(seed_phrase)):
        output, hidden = model(inp, hidden)

        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        seed_phrase += tokens[top_i]
        inp = top_i
        

    return seed_phrase

In [12]:
all_losses = []
loss_avg = 0.

for epoch in tqdma(range(1, num_epochs + 1)):
    batch = next(generate_chunk())
    inps, targets = batch[:, :-1], batch[:, 1:]
    loss = train_char_rnn(inps, targets)
    loss /= batch_size
    loss_avg += loss
    if epoch % print_every == 0:
        print('epoch #{} Loss {} lr [{}]'.format(epoch, loss, lr_scheduler.get_last_lr()))
        print(generate_sample(model, 'мой дядя самых честных правил', 100, temperature=0.8), '\n')

    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0.

  0%|          | 0/2100 [00:00<?, ?it/s]

epoch #100 Loss 0.008194323182106018 lr [[0.01]]
мой дядя самых честных правили.



xv

ке шрады
ей доль:
тот,
она вздо белья
на придет сторастой,
ве 

epoch #200 Loss 0.007186365127563476 lr [[0.01]]
мой дядя самых честных правилею;
как дерещей всять он в остароски всежбов,
не встих их слушать и зан 

epoch #300 Loss 0.007341981530189514 lr [[0.01]]
мой дядя самых честных правилит.



ix

но шум в лишь не молины,
семья не плудельный зарова.



vi

 

epoch #400 Loss 0.006633085608482361 lr [[0.01]]
мой дядя самых честных правил,
своевою заберельным вень.
вот полноко полукой было везде…
и мысли дви 

epoch #500 Loss 0.006685490608215332 lr [[0.01]]
мой дядя самых честных правил
и на сам ольга и разлука моему;
больстодом над инется кольцо;
уты тамб 

epoch #600 Loss 0.006652015447616577 lr [[0.01]]
мой дядя самых честных правилить.
она в туманной вечеровится,
четы странице светит страной.



xxxii 

epoch #700 Loss 0.006350469589233398 lr [[0.01]]
мой дядя самых честных правила,
каками 

В качестве иллюстрации ниже доступен график значений функции потерь, построенный в ходе обучения авторской сети (сам код для ее обучения вам и предстоит написать).

Шаблон функции `generate_sample` также доступен ниже. Вы можете как дозаполнить его, так и написать свою собственную функцию с нуля. Не забывайте, что все примеры в обучающей выборке начинались с токена `<sos>`.

Пример текста сгенерированного обученной моделью доступен ниже. Не страшно, что в тексте много несуществующих слов. Используемая модель очень проста: это простая классическая RNN.

In [13]:
print(generate_sample(model, ' мой дядя самых честных правил', max_length=500, temperature=0.8))

 мой дядя самых честных правил
к выскойшели поках, который,
читали отресть мирных с торку
всё толкой меня презренственной
и что запоздаре верет.
я на что не почуи поклонный
и слиса челянов;
хоть он в смотров беседыт.
спит. памятили становал,
на чуть резям наследстверь и ленский слышит…
при разумавстверью зато поклонный,
всегда повусили с ней,
из шум с приленной перебила
и потом
ее снута, он иль старимый
перестал бог летих трепет
лиептет отвеке новы,
с девствою туполи давь
оно лека тень проходит 


### Сдача задания
Сгенерируйте десять последовательностей длиной 500, используя строку ' мой дядя самых честных правил'. Температуру для генерации выберите самостоятельно на основании визуального качества генериуремого текста. Не забудьте удалить все технические токены в случае их наличия.

Сгенерированную последовательность сохрание в переменную `generated_phrase` и сдайте сгенерированный ниже файл в контест.

In [14]:
seed_phrase = ' мой дядя самых честных правил'

In [15]:
generated_phrases = [
     generate_sample(
         model,
         ' мой дядя самых честных правил',
         max_length=500,
         temperature=1.
     ).replace('<sos>', '')
     for _ in range(10)
 ]
    
# For example:

# generated_phrases = [
#     generate_sample(
#         model,
#         ' мой дядя самых честных правил',
#         max_length=500,
#         temperature=1.
#     ).replace('<sos>', '')
#     for _ in range(10)
# ]

In [16]:
print(generated_phrases)

[' мой дядя самых честных правился,\nгде жела. после…» —\n«в модым пряду зовой польо,\nвыпыл писью трепет ей.\nкака он вперценьем. а не счастьяну.\n\n\n\nxxx\n\nтаков евгеник лучаю,\nдовольный года, с патуменный,\nтатьяна прохожда мужкет;\nо уж элигок плол онегин,\nвозврательный волшебной учец\nот, нупе: он ворвов пустынный,\nсадит с к еще не главо тугом раздели\nзамечанье, горичей.\n\n\n\nxlv\n\nно к ты девожу причуз —\n«да подковский чистала\nзадави трещит он?\n\n\n\nxv\n\nи дар в вольник холомнуто скузный,\nдля прином с племя', ' мой дядя самых честных правил\nмедвед среди душой рекога». —\n«о тот, под нужный вновь чюотому ей.\nумились вы-унивия\nнасладился надрежу подрусь.\nа не показать? ксавить, и вметрает.\n\n\n\nxix\n\nпоключе и в шумен блепал,\nуедвились помнет иногда иль;\nее б романте уходят,\nво можит он песенным картум:\nи, разъезамых давно глядел\nвсем в пилише в тихой тани,\nбы неправный в глядит,\nне. не говориное на —\nлыбкой быйска, чистый лира.\n\n\n\nxxviii\n\nв кок

In [17]:
# do not change the code in the block below
# __________start of block__________

import json
if 'generated_phrases' not in locals():
    raise ValueError("Please, save generated phrases to `generated_phrases` variable")

for phrase in generated_phrases:

    if not isinstance(phrase, str):
        raise ValueError("The generated phrase should be a string")

    if len(phrase) != 500:
        raise ValueError("The `generated_phrase` length should be equal to 500")

    assert all([x in set(tokens) for x in set(list(phrase))]), 'Unknown tokens detected, check your submission!'


submission_dict = {
    'token_to_idx': token_to_idx,
    'generated_phrases': generated_phrases
}

with open('submission_dict.json', 'w') as iofile:
    json.dump(submission_dict, iofile)
print('File saved to `submission_dict.json`')
# __________end of block__________

File saved to `submission_dict.json`


На этом задание завершено. Поздравляем!